In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import plotly.express as px
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# House Prices: Advanced Regression Techniques

This project covers the advanced regression techniques as the dataset contains a lot of variable that are both categorical variables and linear values variables.According to me, this should be project after a kaggler is comfortable with making models and understanding the scores.For my model building , right now I am considering only root mean squared error values and R-squared value.

For further scoring, I was planning to use Adjusted R-squared value.

Details on accuracy for R-squared:

**R-squared** is a goodness-of-fit measure for linear regression models. This statistic indicates the percentage of the variance in the dependent variable that the independent variables explain collectively. R-squared measures the strength of the relationship between your model and the dependent variable on a convenient 0 – 100% scale.



In [ ]:
test_data=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train_data=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
sample_submission=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

backup_data=train_data

Our predicted variable is the **SalePrice** which would be calculated from the several predictor variables that we have.Currently, we don't know anything about any of the details.

There are around 80 columns and 1460 rows.There are a lot of columns that may affect the SalePrice .For that one need to do comprehensive exploration of data.

In [ ]:
test_data.shape

In [ ]:
train_data.shape

In [ ]:
target_column=train_data[['SalePrice']]

In [ ]:
train_data.shape

In [ ]:
# Checking train data
train_data.head()

In [ ]:
# Checking test data
test_data.head()

In [ ]:
# Checking Sample submission data
sample_submission.head()

The pandas describe function gives us a basic information on the values of variable.From understanding of the describe function, The House Sale Prices have a minimum of 34.9 K dollars to 755 K.

In [ ]:
target_column.describe()

As we can see from below, the graph is a normal distribution curve with values between 100K and 300K.

About Gaussian Normal Distribution Curve:

    Gaussian distribution (also known as normal distribution) is a bell-shaped curve, and it is assumed that during any measurement values will follow a normal
    distribution with an equal number of measurements above and below the mean value.
    
    In statistics, the theoretical curve that shows how often an experiment will produce a particular result. The curve is symmetrical and bell shaped, showing that 
    trials will usually give a result near the average, but will occasionally deviate by large amounts. 
    

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(target_column)

### Checking for Null values

Null values can be treated mostly in 2 ways: 
    
       1. Either delete the whole row - This option I usually use when the data records that needs to be removed are not that important and **WOULD NOT** not help 
       in our model building.
       
       2.Imputing the values with mean or a median.I perfer to choose median over mean as I think that mean gets affected by outliers so I prefer median as a way to 
       impute values.

## Removing Outliers 

For removing outliers I came across with the several kernels in which people have ran the below code to remove the outliers.I am still trying to understand as how these were constituted as outliers.Here is the discussion thread.

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/discussion/146728



In [ ]:
# As suggested by many participants, we remove several outliers
train_data.drop(train_data[(train_data['OverallQual']<5) & (train_data['SalePrice']>200000)].index, inplace=True)
train_data.drop(train_data[(train_data['GrLivArea']>4000) & (train_data['SalePrice']<300000)].index, inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [ ]:
def boxplot(column_name,data):
    title='Box plot for column- {0}'.format(column_name)
    #print(title)
    plt.title(title)
    sns.boxplot(x=column_name,data=data)
    details_na='The total number of NA Values for column - {1} is {0}' .format(data[column_name].isna().sum(),column_name)
    print(details_na)
    
def drop_column(data,column_name):
    train_data=data
    if column_name in data.columns:
        train_data=data.drop(columns=[column_name],axis=1)
        return train_data
    else:
        print('Column already removed from dataset Or column not present \n')
        print(' ,'.join (data.columns))
        return train_data
    
def remove_outliers(column_name,data):
    quantile_1=data[column_name].quantile(0.25)
    quantile_3=data[column_name].quantile(0.75)
    iqr=quantile_3-quantile_1
    lower=quantile_1-1.5*iqr
    upper=quantile_1+1.5*iqr
    data[column_name]=data[(data[column_name]>lower)& (data[column_name]<upper)][column_name]
    return data[column_name]

I have divided sets into 3 parts -

    1. Object type variables 
    2. Integer type variables -- The missing values would be added with **median** values
    3. Integer turned Category variables 

In [ ]:
## Convert to category variable
## Dividing the list between object type variables and int variables and then subsetting it

obj_type_variables = [column for column in train_data.columns if train_data[column].dtype in ['object']]
object_data=train_data[obj_type_variables]
object_data_test=test_data[obj_type_variables]

obj_new_list=['MSSubClass','OverallQual','OverallCond','BsmtFullBath','GarageCars','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd'
              ,'Fireplaces','MoSold']
train_data[obj_new_list]=train_data[obj_new_list].apply(lambda column:column.astype('object'))
test_data[obj_new_list]=test_data[obj_new_list].apply(lambda column:column.astype('object'))

category_data=train_data[obj_new_list]
category_data_test=test_data[obj_new_list]

int_type_variables = [column for column in train_data.columns if train_data[column].dtype in ['int64','float64']]
int_data=train_data[int_type_variables]
int_data_test=test_data[int_type_variables[:-1]]

In [ ]:
print(int_type_variables)

In [ ]:
test_data[obj_new_list].isna().sum()

We would be replacing the removing values with median values.I choose median values over mean values as means get affected with outliers while it is not the same case with median values

In [ ]:
object_data_test.isna().sum()

## Replacing NaN values for categorical features

    1. Alley variable contains 2 variables,rest has been replaced with None Variable
    2. For the rest values, I took up the mode values i.e. the higher categorical variables is taken
    3. For some higher NaN values,I will replace it with NA

In [ ]:
object_data['BsmtExposure'].value_counts()


In [ ]:
x=['BsmtHalfBath', 'BsmtFullBath', 'GarageCars']
for i in x:
    print(train_data[i].value_counts())


In [ ]:

object_data['Alley']=object_data['Alley'].fillna("Grvl")
object_data['BsmtQual']=object_data['BsmtQual'].fillna("TA")
object_data['BsmtCond']=object_data['BsmtCond'].fillna("TA")
object_data['BsmtExposure']=object_data['BsmtExposure'].fillna("No")
object_data['BsmtFinType1']=object_data['BsmtFinType1'].fillna("Unf")
object_data['BsmtFinType2']=object_data['BsmtFinType2'].fillna("Unf")
object_data['Electrical']=object_data['Electrical'].fillna("SBrkr")
object_data['FireplaceQu']=object_data['FireplaceQu'].fillna("Gd")
object_data['GarageType']=object_data['GarageType'].fillna("Attchd")
object_data['GarageFinish']=object_data['GarageFinish'].fillna("Unf")
object_data['GarageQual']=object_data['GarageQual'].fillna("TA")
object_data['GarageCond']=object_data['GarageCond'].fillna("TA")
object_data['PoolQC']=object_data['GarageCond'].fillna("TA")
object_data['Fence']=object_data['Fence'].fillna("MnPrv")
object_data['MiscFeature']=object_data['MiscFeature'].fillna("Shed")

train_data['BsmtHalfBath']=train_data['BsmtHalfBath'].fillna("0")
train_data['BsmtFullBath']=train_data['BsmtFullBath'].fillna("0")
train_data['GarageCars']=train_data['GarageCars'].fillna("2")



object_data_test['Alley']=object_data_test['Alley'].fillna("Grvl")
object_data_test['BsmtQual']=object_data_test['BsmtQual'].fillna("TA")
object_data_test['BsmtCond']=object_data_test['BsmtCond'].fillna("TA")
object_data_test['BsmtExposure']=object_data_test['BsmtExposure'].fillna("No")
object_data_test['BsmtFinType1']=object_data_test['BsmtFinType1'].fillna("Unf")
object_data_test['BsmtFinType2']=object_data_test['BsmtFinType2'].fillna("Unf")
object_data_test['Electrical']=object_data_test['Electrical'].fillna("SBrkr")
object_data_test['FireplaceQu']=object_data_test['FireplaceQu'].fillna("Gd")
object_data_test['GarageType']=object_data_test['GarageType'].fillna("Attchd")
object_data_test['GarageFinish']=object_data_test['GarageFinish'].fillna("Unf")
object_data_test['GarageQual']=object_data_test['GarageQual'].fillna("TA")
object_data_test['GarageCond']=object_data_test['GarageCond'].fillna("TA")
object_data_test['PoolQC']=object_data_test['GarageCond'].fillna("TA")
object_data_test['Fence']=object_data_test['Fence'].fillna("MnPrv")
object_data_test['MiscFeature']=object_data_test['MiscFeature'].fillna("Shed")

test_data['BsmtHalfBath']=test_data['BsmtHalfBath'].fillna("0")
test_data['BsmtFullBath']=test_data['BsmtFullBath'].fillna("0")
test_data['GarageCars']=test_data['GarageCars'].fillna("2")


#object_data['MiscFeature'].value_counts()
#train_data_X_int=train_data_X_int.fillna(train_data_X_int.median())


In [ ]:
test_data['GarageCars'].isna().sum()

In [ ]:
object_data.isna().sum()

In [ ]:
object_data.shape

## Preprocessing Data

I used basic preprocessing i.e. filling missing values, didnt remove any outliers and created a base model which gave out a score of around 85%.I am now trying to build a model after properly cleaning the data i.e. removing outliers , filling NA's with proper values.

I referenced some of the top voted kernels therefore I hope that after I clean the data, the result would be a little better.I will comment out on the box plots after I have visualised on it.

### Working on integer related data

I am making box job and trying to get an understanding if we have need to clear any outliers.Simultaneously, I will fill the missing values.For that I have created a function as well.

In [ ]:
boxplot('SalePrice',target_column)

As understood from the target column, the box plot depicts some of the outliers.

In [ ]:
corr_data=int_data.corr()

## Heat Map for the data

Heat map helps to find out the correlation between variables.The corr() function uses Pearson Correlation.I used heatmap because I wanted to check if a high correlation is affected by variables to the target variables.



In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(corr_data,annot=True)


Understanding from the above correlation map, I am going to take on some of the variables that have got high correlation and understand the flow information.Below is the relation of variables between each other

            |Column Name               |Correlated Columns with high correlation|
            
            |--------------------------|----------------------------------------|
            
            |MSSubClass                |LotFrontage,TotalBsmtSF,1stFlrSF        |
            |LotFrontage               |  NA                                    |
            |LotArea                   |  NA                                    |
            |OverallQual               | GrLivArea                              |
            |OverallCond               | GarageYrBlt                            |
            |YearBuilt                 |EnclosedPorch,GarageYrBlt,YearRemodAdd  |
            |YearRemodAdd              |GarageYrBlt                             |
            |MasVnrArea                |  NA                                    |
            |BsmtFinSF1                |BsmtFullBath                            |
            |BsmtFinSF2                |  NA                                    |
            |BsmtUnfSF                 |BsmtFullBath                            |
            |TotalBsmtSF               |1stFlrSF                                |
            |1stFlrSF                  |  NA                                    |
            |2ndFlrSF                  | GrLivArea                              |
            |LowQualFinSF              |  NA                                    |
            |GrLivArea                 |  NA                                    |
            |BsmtFullBath              |  NA                                    |
            |BsmtHalfBath              |  NA                                    |
            |FullBath                  |  NA                                    |
            |HalfBath                  |  NA                                    |
            |BedroomAbvGr              |Fireplaces                              |
            |KitchenAbvGr              |  NA                                    |
            |TotRmsAbvGrd              |  NA                                    |
            |Fireplaces                |  NA                                    |
            |GarageYrBlt               | EnclosedPorch                          |
            |GarageCars                | GarageArea                             |
            |GarageArea                |  NA                                    |
            |WoodDeckSF                |  NA                                    |
            |OpenPorchSF               |  NA                                    |
            |EnclosedPorch             |  NA                                    |
            |3SsnPorch                 |  NA                                    |
            |ScreenPorch               |  NA                                    |
            |PoolArea                  |  NA                                    |
            |MiscVal                   |  NA                                    |
            |MoSold                    |  NA                                    |
            |YrSold                    |  NA                                    |
            


In [ ]:
pair_plot_data_inbetween_variables=train_data[['MSSubClass',
          'LotFrontage',
         'TotalBsmtSF',
         'OverallQual',
         'GrLivArea',
         'OverallCond',
         'YearBuilt',
         'EnclosedPorch',
         'GarageYrBlt',
         'YearRemodAdd',
         'BsmtFinSF1',
         'BsmtFullBath',
         'BsmtUnfSF',
         '1stFlrSF',
         '2ndFlrSF',
         'BedroomAbvGr',
         'Fireplaces',
         'GarageCars',
         'GarageArea'
         ]]



In [ ]:
pair_plot_data_inbetween_variables.head()

After shortlisting the variables, I built a pair plot between the variables which have either a high positive correlation or a negative correlation.

In [ ]:
#plt.figure(figsize=(30,30))
#sns.pairplot(pair_plot_data_inbetween_variables)

In [ ]:
pair_plot_data_target_variable=train_data[[
    'SalePrice',
    'OverallQual',
    'TotalBsmtSF',
    '1stFlrSF',
    'GrLivArea',
    'GarageCars',
    'GarageArea'
]]

Below pair plot is created with respect to the SalePrice(target variable) with other variables.

In [ ]:
sns.pairplot(pair_plot_data_target_variable)

In [ ]:
## Creating graphs between SalePrice and ['GarageArea','GrLivArea','1stFlrSF','TotalBsmtSF']

plt.figure(figsize=(9,9))
sns.relplot(x="SalePrice",
                y="GrLivArea",
                col="BldgType",
                hue="OverallQual",
                kind="scatter",
                height=10,
                aspect=0.3,
                data=train_data)

The above graph is between SalePrice and GrLivArea(Above grade (ground) living area square feet), divided across Building Type and Overall Quality of values.

As understood from the above the records, building type for 1Fam has a large number of records and it is a positive trend i.e. with the increase in SalePrice there is a increase in Quality.

In [ ]:
plt.figure(figsize=(9,9))
sns.relplot(x="SalePrice",
                y="GarageArea",
                col="GarageCars",
                hue="GarageQual",
                kind="scatter",
                height=10,
                aspect=0.3,
                data=train_data)

The above graph is between SalePrice and Garage Area.The Graph is spreaded for Garage Cars as 3 while for GarageCars=2 the no. of records is high as compared to the other records.Also, the no. of records for Average Garage quality is high.

In [ ]:
fig = px.scatter(train_data, x="SalePrice", y="1stFlrSF", color="OverallQual")
fig.show()

The above graph shows the flow between SalePrice and 1stFlrSF.The Overall Quality would show the flow of the SalePrice values.

In [ ]:
fig = px.scatter(train_data, x="SalePrice", y="TotalBsmtSF", color="OverallQual")
fig.show()

## Model Building

For Model building, I am first creating a base model that considers all variables for our predictor variables and predicted variable is our SalePrice.


In [ ]:
#print(obj_type_variables)
#print(int_type_variables)
int_selected=int_type_variables[1:-1]

In [ ]:
train_data[obj_type_variables].shape

In [ ]:
test_data[obj_type_variables].shape

In [ ]:
print(train_data[obj_type_variables].shape[0])
print(test_data[obj_type_variables].shape)

In [ ]:
obj_type_variables=obj_type_variables+obj_new_list
obj_type_variables=list(set(obj_type_variables))

train_data_X_obj = train_data[obj_type_variables].apply(lambda column: column.astype('category').cat.codes)
test_data_X_obj=test_data[obj_type_variables].apply(lambda column: column.astype('category').cat.codes) 


#train_data_X_obj = train_data[obj_type_variables].apply(lambda column: column.astype('category'))
#test_data_X_obj=test_data[obj_type_variables].apply(lambda column: column.astype('category'))

#train_data_X_obj = pd.get_dummies(train_data_X_obj)
#test_data_X_obj = pd.get_dummies(test_data_X_obj)

In [ ]:
#combined_data = pd.concat((train_data[obj_type_variables], test_data[obj_type_variables]), sort=False).reset_index(drop=True)
#combined_data_dummies=pd.get_dummies(combined_data)

In [ ]:
#train_data_X_obj=combined_data_dummies.iloc[:train_data[obj_type_variables].shape[0]].reset_index(drop=True)
#test_data_X_obj = combined_data_dummies.iloc[train_data[obj_type_variables].shape[0]:].reset_index(drop=True)

In [ ]:
print(train_data_X_obj.shape)
print(test_data_X_obj.shape)

In [ ]:
train_data_X_int=train_data[int_selected]
test_data_X_int=test_data[int_selected]

In [ ]:
## Checking for missing values and imputing them
print('Checking for categorical variables for missing values')
print(train_data_X_obj.isna().sum())
print('Checking for integer variables for missing values')

print(train_data_X_int.isna().sum())


In [ ]:
train_data_X_int=train_data_X_int.fillna(train_data_X_int.median())
test_data_X_int=test_data_X_int.fillna(train_data_X_int.median())

In [ ]:
# scaler=StandardScaler()
#scaler=RobustScaler()
#scaler=MinMaxScaler()
#train_data_X_int=pd.DataFrame(scaler.fit_transform(train_data_X_int),columns=train_data_X_int.columns)
#test_data_X_int=pd.DataFrame(scaler.fit_transform(test_data_X_int),columns=test_data_X_int.columns)

In [ ]:
train_Y=train_data['SalePrice']

In [ ]:
train_X=train_data_X_int.merge(train_data_X_obj,left_index=True,right_index=True).reset_index(drop=True)
test_X=test_data_X_int.merge(test_data_X_obj,left_index=True,right_index=True).reset_index(drop=True)

In [ ]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)

In [ ]:
test_X.columns[test_X.isna().any()].tolist()

In [ ]:
print(train_X.shape)
print(train_Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_Y,test_size = .3, random_state=0)

I divided dataset into 70-30 ratio so that we can test our scenarios.

In [ ]:

rf = RandomForestRegressor()
params = {"max_depth":[15,20,25], "n_estimators":[24,30,36]}
rf_reg = GridSearchCV(rf, params, cv = 10, n_jobs =10)
rf_reg.fit(X_train, Y_train)
print(rf_reg.best_estimator_)
best_estimator=rf_reg.best_estimator_
y_pred_test = best_estimator.predict(X_test)



In [ ]:
print('Mean Square Error test = ' + str(mean_squared_error(Y_test, y_pred_test,squared=False)))
print('Root Mean Square Error test = ' + str((mean_squared_error(np.log(Y_test), np.log(y_pred_test),squared=False))))

print ('R2 square '+str(r2_score(Y_test, y_pred_test))) 

In [ ]:
print(len(X_train.columns))
print(len(set(X_train.columns)))



In [ ]:
#from xgboost import XGBRegressor
#xgb_regressor = XGBRegressor(learning_rate=0.02,n_estimators = 2400,max_depth=3, min_child_weight=0.01,gamma=0, subsample=0.7,
#                                     colsample_bytree=0.7,objective='reg:linear', nthread=-1, scale_pos_weight=1, seed=27,reg_alpha=0.00006)
#xgb_regressor.fit(X_train,Y_train)
#y_pred_test=xgb_regressor.predict(X_test)
#
#mean_squared_error(Y_test, y_pred_test)
#

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor(n_estimators=1992, learning_rate=0.03, max_depth=3, max_features='sqrt', min_samples_leaf=15, min_samples_split=8, loss='huber', random_state =42)
gb_regressor.fit(X_train,Y_train)
y_pred_test=gb_regressor.predict(X_test)



In [ ]:
print('Mean Square Error test = ' + str(mean_squared_error(Y_test, y_pred_test,squared=False)))
print('Root Mean Square Error test = ' + str((mean_squared_error(np.log(Y_test), np.log(y_pred_test),squared=False))))

print ('R2 square '+str(r2_score(Y_test, y_pred_test)))

In [ ]:
#pred_test = best_estimator.predict(test_X)
pred_test = gb_regressor.predict(test_X)

In [ ]:
sample_submission=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
submission=pd.DataFrame({"Id":sample_submission['Id'],"SalePrice":pred_test})
submission.to_csv('submission.csv',index=False)

In [ ]:
sample_submission.head()

 I have taken the references from the below sites and kernels.Thanks a lot guys!!!!!
 
 
     https://stats.idre.ucla.edu/spss/faq/coding-systems-for-categorical-variables-in-regression-analysis/
    
    https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html
    
    https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
    
    https://www.kaggle.com/janvichokshi/advanced-regression-techniques
    
    https://statisticsbyjim.com/regression/interpret-r-squared-regression/
    
    https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
    
    https://www.kaggle.com/agehsbarg/top-10-0-10943-stacking-mice-and-brutal-force

P.S.
I making more changes to this kernel I think I can make better models by using other variants and doing feature selections.
